In [47]:
# Importação de bibliotecas
import pandas as pd
import numpy as np

In [3]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Ouro

##### Lavouras Temporárias

In [108]:
df_spd = (
    pd.read_parquet(f'{raiz}/prata_SPD.parquet')
    .reset_index()
)

In [122]:
df_temp = (
    pd.read_parquet(f'{raiz}/prata_sidra_tab1612_lavoura_temporaria_area_colhida.parquet')
    .query('data >= 2000')
    .reset_index()
    .assign(geocodigo_uf = lambda df: (df.geocodigo// 100000))
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

In [110]:
# Soja == 2713
(
    df_temp
    .query('cultura == 2713')
    .reset_index()
    .merge(
        df_spd,
        how='left',
        on=['geocodigo', 'data']
    )
    .assign(**{
        'Soja - Convencional - Monocultura': lambda x: np.where(x.data >= pd.to_datetime('2005'), ((x.area/1000000) - x.valor)*0.5, (x.area/1000000)*0.5),
        'Soja - Convencional - Sucessão': lambda x: np.where(x.data >= pd.to_datetime('2005'), (x.area/1000000)*0.5 - x.valor, (x.area/1000000)*0.5),
        'Soja - Plantio Direto': lambda x: np.where(x.data >= pd.to_datetime('2005'), (x.area/1000000)*0.7 - x.valor, (x.area/1000000)*0.7),
        'Soja - SPD bem feito': lambda x: np.where(x.data >= pd.to_datetime('2005'), (x.area/1000000)*0.3 - x.valor, (x.area/1000000)*0.3)
    })
    .groupby(['data', 'cultura']).agg({'Soja - Convencional - Monocultura':sum})
    # .drop(columns=(['cultura', 'area', 'valor']))
    # .melt(
    #     id_vars=['geocodigo', 'data'],
    #     var_name='cultura',
    #     value_name='valor'
    # )
    # .dropna(subset='valor')
    .convert_dtypes()
) # Pedir orientação de como utilizar o SPD

,,Soja - Convencional monocultura
data,cultura,
2003-01-01,2713,9.262384
2004-01-01,2713,10.769495
2005-01-01,2713,-0.363787
2006-01-01,2713,-1.668053
2007-01-01,2713,-2.62786
2008-01-01,2713,-2.692508
2009-01-01,2713,-2.867866
2010-01-01,2713,-2.61172
2011-01-01,2713,-2.86774


In [100]:
# Milho == 2711
(
    df_temp
    .query('cultura == 2711')
    .reset_index()
    .assign(**{
        'Milho - 1a. safra - monocultura': lambda x: round((x.area/1000000)*0.06, 3),
        'Milho - 1a. safra': lambda x: round((x.area/1000000)*0.19, 3),
        'Milho - 2a. safra (verão)': lambda x: round((x.area/1000000)*0.16, 3),
        'Milho - 2a. safra + braquiária': lambda x: round((x.area/1000000)*0.3, 3),
        'Milho - 2a. safra - SPD': lambda x: round((x.area/1000000)*0.3, 3)
    })
    #.groupby(['data', 'cultura']).agg({'Milho - 1a. safra - monocultura':sum})
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,11,2003-01-01,Milho - 1a. safra - monocultura,0.006
1,11,2004-01-01,Milho - 1a. safra - monocultura,0.007
2,11,2005-01-01,Milho - 1a. safra - monocultura,0.007
3,11,2006-01-01,Milho - 1a. safra - monocultura,0.008
4,11,2007-01-01,Milho - 1a. safra - monocultura,0.007
...,...,...,...,...
2695,53,2018-01-01,Milho - 2a. safra - SPD,0.019
2696,53,2019-01-01,Milho - 2a. safra - SPD,0.019
2697,53,2020-01-01,Milho - 2a. safra - SPD,0.018
2698,53,2021-01-01,Milho - 2a. safra - SPD,0.018


In [101]:
# Trigo == 2716
(
    df_temp
    .query('cultura == 2716')
    .reset_index()
    .assign(**{
        'Trigo - Convencional': lambda x: round((x.area/1000000)*0.1, 4),
        'Trigo - Plantio Direto (no-till)': lambda x: round((x.area/1000000)*0.5, 3),
        'Trigo - Cultivo mínimo, SPD bem feito': lambda x: round((x.area/1000000)*0.4, 3)
    })
    #.groupby(['data', 'cultura']).agg({'Trigo - Plantio Direto (no-till)':sum})
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,13,2022-01-01,Trigo - Convencional,0.0
1,23,2020-01-01,Trigo - Convencional,0.0
2,29,2003-01-01,Trigo - Convencional,0.0001
3,29,2004-01-01,Trigo - Convencional,0.0001
4,29,2005-01-01,Trigo - Convencional,0.0
...,...,...,...,...
541,53,2018-01-01,"Trigo - Cultivo mínimo, SPD bem feito",0.001
542,53,2019-01-01,"Trigo - Cultivo mínimo, SPD bem feito",0.001
543,53,2020-01-01,"Trigo - Cultivo mínimo, SPD bem feito",0.001
544,53,2021-01-01,"Trigo - Cultivo mínimo, SPD bem feito",0.001


In [102]:
# Algodão Herbáceo == 2689
(
    df_temp
    .query('cultura == 2698')
    .reset_index()
    .assign(**{
        'Algodão - Convencional - morro abaixo': lambda x: round((x.area/1000000)*0.4, 4),
        'Algodão - Plano/ondulado, conservado': lambda x: round((x.area/1000000)*0.6, 4)
    })
    #.groupby(['data', 'cultura']).agg({'Algodão - Convencional - morro abaixo':sum})
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,41,2003-01-01,Algodão - Convencional - morro abaixo,0.0003
1,41,2004-01-01,Algodão - Convencional - morro abaixo,0.0003
2,41,2005-01-01,Algodão - Convencional - morro abaixo,0.0003
3,41,2006-01-01,Algodão - Convencional - morro abaixo,0.0001
4,41,2007-01-01,Algodão - Convencional - morro abaixo,0.0003
...,...,...,...,...
115,50,2013-01-01,"Algodão - Plano/ondulado, conservado",0.0007
116,50,2014-01-01,"Algodão - Plano/ondulado, conservado",0.0003
117,50,2015-01-01,"Algodão - Plano/ondulado, conservado",0.0001
118,50,2016-01-01,"Algodão - Plano/ondulado, conservado",0.0002


In [105]:
# Mandioca == 2708
(
    df_temp
    .query('cultura == 2698')
    .reset_index()
    .assign(**{
        'Mandioca - Convencional - solo coberto (entressafra)': lambda x: round(x.area/1000000, 4)
    })
    #.groupby(['data', 'cultura']).agg({'Mandioca - Convencional - solo coberto (entressafra)':sum})
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,41,2003-01-01,Mandioca - Convencional - solo coberto (entres...,0.0007
1,41,2004-01-01,Mandioca - Convencional - solo coberto (entres...,0.0006
2,41,2005-01-01,Mandioca - Convencional - solo coberto (entres...,0.0007
3,41,2006-01-01,Mandioca - Convencional - solo coberto (entres...,0.0003
4,41,2007-01-01,Mandioca - Convencional - solo coberto (entres...,0.0007
5,41,2008-01-01,Mandioca - Convencional - solo coberto (entres...,0.0016
6,41,2009-01-01,Mandioca - Convencional - solo coberto (entres...,0.0006
7,41,2010-01-01,Mandioca - Convencional - solo coberto (entres...,0.0004
8,41,2011-01-01,Mandioca - Convencional - solo coberto (entres...,0.0007
9,41,2012-01-01,Mandioca - Convencional - solo coberto (entres...,0.0011


In [113]:
# Cereais diversos:
# Arroz == 2692
# Aveia == 2693
# Centeio == 2698
# Cevada == 2699
# Feijão == 2702
# Girassol == 109179
# Mamona == 2707
# Sorgo == 2714
# Triticale == 109180
(
    df_temp
    .query('cultura in (2692, 2693, 2698, 2699, 2702, 109179, 2714, 2707, 109180)')
    .groupby(['geocodigo', 'data']).agg({'area': sum})
    .reset_index()
    .assign(**{
        'Aveia (cereais) - Convencional - Sem Cobertura': lambda x: round((x.area/1000000)*0.3, 4),
        'Aveia (cereais) - Convencional': lambda x: round((x.area/1000000)*0.4, 4),
        'Aveia (cereais) - SPD': lambda x: round((x.area/1000000)*0.3, 4)
    })
    #.groupby('data').agg({'Aveia (cereais) - Convencional - Sem Cobertura':sum})
    .drop(columns=('area'))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,11,2003-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0354
1,11,2004-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0436
2,11,2005-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0476
3,11,2006-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0399
4,11,2007-01-01,Aveia (cereais) - Convencional - Sem Cobertura,0.0397
...,...,...,...,...
1615,53,2018-01-01,Aveia (cereais) - SPD,0.007
1616,53,2019-01-01,Aveia (cereais) - SPD,0.0057
1617,53,2020-01-01,Aveia (cereais) - SPD,0.0087
1618,53,2021-01-01,Aveia (cereais) - SPD,0.0058


In [116]:
# Cana-de-açúcar == 2696
(
    df_temp
    .query('cultura == 2696')
    .reset_index()
    .assign(**{
        'Cana - Planta somente': lambda x: round((x.area/1000000)*0.2, 4),
        'Cana - Planta + 4 cana-soca (5 colheitas)': lambda x: round((x.area/1000000)*0.8, 4)
    })
    #.groupby(['data', 'cultura']).agg({'Cana - Planta + 4 cana-soca (5 colheitas)':sum})
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)


,geocodigo,data,cultura,valor
0,11,2003-01-01,Cana - Planta somente,0.0001
1,11,2004-01-01,Cana - Planta somente,0.0001
2,11,2005-01-01,Cana - Planta somente,0.0001
3,11,2006-01-01,Cana - Planta somente,0.0003
4,11,2007-01-01,Cana - Planta somente,0.0002
...,...,...,...,...
1075,53,2018-01-01,Cana - Planta + 4 cana-soca (5 colheitas),0.0002
1076,53,2019-01-01,Cana - Planta + 4 cana-soca (5 colheitas),0.0002
1077,53,2020-01-01,Cana - Planta + 4 cana-soca (5 colheitas),0.0002
1078,53,2021-01-01,Cana - Planta + 4 cana-soca (5 colheitas),0.0002


##### Lavouras Permanentes

In [127]:
df_perm = (
    pd.read_parquet(f'{raiz}/prata_sidra_tab1613_lavoura_permanente_area_colhida.parquet')
    .query('data >= 2000')
    .reset_index()
    .assign(geocodigo_uf = lambda df: (df.geocodigo// 100000))
    .groupby(['geocodigo_uf', 'data', 'cultura'])
    .agg({'area':sum})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
    .convert_dtypes()
)

In [124]:
# Café == 2723
(
    df_perm
    .query('cultura == 2723')
    .reset_index()
    .assign(**{
        'Café - Convencional + cobertura do solo': lambda x: round(x.area/1000000, 4)
    })
    #.groupby(['data', 'cultura']).agg({'Café - Convencional + cobertura do solo':sum})
    .drop(columns=(['cultura', 'area']))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
)

,geocodigo,data,cultura,valor
0,11,2000-01-01,Café - Convencional + cobertura do solo,0.1989
1,11,2001-01-01,Café - Convencional + cobertura do solo,0.2229
2,11,2002-01-01,Café - Convencional + cobertura do solo,0.141
3,11,2003-01-01,Café - Convencional + cobertura do solo,0.1885
4,11,2004-01-01,Café - Convencional + cobertura do solo,0.1712
...,...,...,...,...
395,53,2018-01-01,Café - Convencional + cobertura do solo,0.0003
396,53,2019-01-01,Café - Convencional + cobertura do solo,0.0004
397,53,2020-01-01,Café - Convencional + cobertura do solo,0.0004
398,53,2021-01-01,Café - Convencional + cobertura do solo,0.0004


In [134]:
# Frutíferas:
# Abacate == 2717
# Abacaxi == 2688 - lav. temporária
# Açaí == 45981
# Banana == 2720
# Caqui == 2724
# Castanha de caju == 2725
# Figo == 2730
# Goiaba == 2731
# Laranja == 2733
# Limão == 2734
# Maçã == 2735
# Mamão == 2736
# Manga == 2737
# Maracujá == 2738
# Marmelo == 2739
# Melancia == 2709 - lav. temporária
# Melão == 2710 - lav. temporária
# Uva == 2748
# Pera == 2741
# Pêssego == 2742
# Tangerina == 2745
(
    pd.concat([
        df_perm
        .query('cultura in (2717, 45981, 2720, 2724, 2725, 2730, 2731, 2733, 2734, 2735, 2736, 2737, 2738, 2739, 2748, 2741, 2742, 2745)'),
        df_temp
        .query('cultura in (2688, 2709, 2710)')
    ])
    .groupby(['geocodigo', 'data']).agg({'area': sum})
    .reset_index()
    .assign(**{
         'Frutíferas - Sem cobertura do solo - Solo nu': lambda x: round((x.area/1000000), 3)
    })
    .groupby('data').agg({'Frutíferas - Sem cobertura do solo - Solo nu':sum})
    .drop(columns=('area'))
    .melt(
        id_vars=['geocodigo', 'data'],
        var_name='cultura',
        value_name='valor'
    )
    .dropna(subset='valor')
    .convert_dtypes()
) # Perguntar por que os últimos anos usou apenas 4 cultivos, impactando no resultado final

,Frutíferas - Sem cobertura do solo - Solo nu
data,
2000-01-01,2.367
2001-01-01,2.309
2002-01-01,2.342
2003-01-01,2.535
2004-01-01,2.515
2005-01-01,2.513
2006-01-01,2.564
2007-01-01,2.622
2008-01-01,2.63


##### Pastagens